In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [26]:
from keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization, CuDNNLSTM

In [27]:
import tensorflow as tf 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

mnist=tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [28]:
print(x_train.shape[1:])
print(x_train[0].shape)
x_train=x_train/255.0
x_test=x_test/255.0

(28, 28)
(28, 28)


In [29]:
model=Sequential()
model.add(LSTM(128,input_shape=(x_train.shape[1:]),activation='relu',return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 28, 128)           80384     
_________________________________________________________________
dropout_12 (Dropout)         (None, 28, 128)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_13 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_14 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)               

In [30]:
model.fit(x_train,y_train,epochs=3,validation_data=(x_test,y_test))

Epoch 1/3
1875/1875 [==============================] - 85s 44ms/step - loss: 0.5478 - accuracy: 0.8251 - val_loss: 0.1634 - val_accuracy: 0.9523
Epoch 2/3
1875/1875 [==============================] - 82s 44ms/step - loss: 0.1550 - accuracy: 0.9589 - val_loss: 0.0910 - val_accuracy: 0.9737
Epoch 3/3
1875/1875 [==============================] - 83s 44ms/step - loss: 0.1144 - accuracy: 0.9713 - val_loss: 0.0737 - val_accuracy: 0.9791


In [31]:
y_pred=model.predict(x_test)
y_pred=[np.argmax(i) for i in y_pred]
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred,y_test))

0.9791


In [32]:
model=Sequential()
model.add(CuDNNLSTM(128,input_shape=(x_train.shape[1:]),return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(128))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_4 (CuDNNLSTM)     (None, 28, 128)           80896     
_________________________________________________________________
dropout_15 (Dropout)         (None, 28, 128)           0         
_________________________________________________________________
cu_dnnlstm_5 (CuDNNLSTM)     (None, 128)               132096    
_________________________________________________________________
dropout_16 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                4128      
_________________________________________________________________
dropout_17 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)               

In [33]:
model.fit(x_train,y_train,epochs=3,validation_data=(x_test,y_test))

Epoch 1/3
1875/1875 [==============================] - 12s 6ms/step - loss: 0.4071 - accuracy: 0.8759 - val_loss: 0.1246 - val_accuracy: 0.9620
Epoch 2/3
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1249 - accuracy: 0.9668 - val_loss: 0.0971 - val_accuracy: 0.9716
Epoch 3/3
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0873 - accuracy: 0.9770 - val_loss: 0.0546 - val_accuracy: 0.9841


In [34]:
y_pred=model.predict(x_test)
y_pred=[np.argmax(i) for i in y_pred]
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred,y_test))

0.9841
